# GTEx rare variants analysis

## Imports

In [1]:
import polars as pl
import matplotlib.pyplot as plt
from pathlib import Path
import pandas as pd
import statsmodels.api as sm
import plotnine as pn
import pyranges as pr

%load_ext autoreload
%autoreload 2

In [2]:
plt.rcParams['figure.dpi'] = 150

In [3]:
pl.Config.set_fmt_str_lengths(100)

polars.config.Config

In [4]:
pn.theme_set(pn.theme_bw())
pn.theme_update(dpi=150)

In [19]:
# tss variant path
pas_variant_path = '/s/project/promoter_prediction/kipoi_expression_prediction/variant_tables/apa_variants_2000_2000.parquet'
# GTEx variants
variant_path = '/s/project/rep/processed/training_results_v16/gtex_v8_old_dna/private_variants.parquet/rare_variants.vcf.parquet/**/*.parquet'
# abexp benchmark dataset
gtex_benchmark_with_annotation_path = "/s/project/rep/processed/training_results_v16/gtex_benchmark_with_annotation.parquet/*.parquet"

## Analysis

We are only loading ensembl canonical transcripts for this analysis. So there is 1 transcript per gene!

In [20]:
pas_variants_ldf = (pl.scan_parquet(Path(pas_variant_path)).
                       select(pl.col(['gene_id', 'pas_id', 'cse_pos']), 
                              pl.col('strand').cast(pl.Enum(['-', '+'])),
                              pl.col(['chrom', 'variant_start', 'variant_end', 'ref', 'alt',])).
                       rename({'gene_id': 'gene', 'pas_id': 'pas'}).
                       with_columns(pl.col('gene').str.replace(r'([^\.]+)\..+$', "${1}").alias('gene')))


# It is possible that a gene comes multiple times (different versions)

In [21]:
variant_ldf = (pl.scan_parquet(variant_path, hive_partitioning=True)
               .select(['sampleId', 'chrom', 'start', 'end', 'ref', 'alt'])
               .rename({'sampleId': 'individual','start': 'variant_start','end': 'variant_end'}))

In [22]:
benchmark_columns = pl.scan_parquet(gtex_benchmark_with_annotation_path).columns
feature_columns = [c for c in benchmark_columns if '@' in c]
training_benchmark_ldf = (pl.scan_parquet(gtex_benchmark_with_annotation_path)
                          .select(['gene', 'individual', 'tissue', 
                                   'FDR', 'zscore', 'AbExp',
                                   *feature_columns,
                                  ])
                          .unique()
                          .with_columns(outlier_state=(pl.when(pl.col('FDR') > 0.05)
                                                       .then(pl.lit('normal'))
                                                       .otherwise(
                                                           pl.when(pl.col('zscore') > 0)
                                                           .then(pl.lit('overexpressed'))
                                                           .otherwise(
                                                               pl.when(pl.col('zscore') < 0)
                                                               .then(pl.lit('underexpressed'))
                                                               # this should never be the case
                                                               .otherwise(pl.lit('CHECK'))
                                                           ))).cast(pl.Enum(['underexpressed', 'normal', 'overexpressed'])),
                                       AbExp=-pl.col('AbExp')))

/scratch/tmp/tsi/ipykernel_3433569/637943636.py:1: PerformanceWarning: Determining the column names of a LazyFrame requires resolving its schema, which is a potentially expensive operation. Use `LazyFrame.collect_schema().names()` to get the column names without this warning.


In [23]:
feature_columns

['expected_expr@theta',
 'splice_ensemble@AbSplice',
 'splice_ensemble@MMSplice_SpliceMap_Psi_ref',
 'splice_ensemble@SpliceAI',
 'vep@cadd_raw.max',
 'vep@LoF_HC.proportion',
 'vep@transcript_ablation.proportion',
 'vep@stop_gained.proportion',
 'vep@frameshift_variant.proportion',
 'vep@coding_sequence_variant.proportion',
 'vep@missense_variant.proportion',
 'vep@inframe_deletion.proportion',
 'vep@inframe_insertion.proportion',
 'vep@stop_lost.proportion',
 'vep@3_prime_UTR_variant.proportion',
 'vep@5_prime_UTR_variant.proportion',
 'vep@NMD_transcript_variant.proportion',
 'vep@NMD_escaping_variant.proportion',
 'vep@start_lost.proportion',
 'vep@splice_donor_variant.proportion',
 'vep@splice_acceptor_variant.proportion',
 'vep@splice_region_variant.proportion']

### What is the enformer variant-effect-score distribution around the TSS?

In [24]:
pas_variants_ldf = (pas_variants_ldf
                    .with_columns(cse_distance=(pl.when(pl.col('strand') == '+')
                                                .then(pl.col('variant_start') - pl.col('cse_pos'))            
                                                .otherwise(pl.col('cse_pos') - pl.col('variant_start'))))
                    .with_columns(absolute_cse_distance=pl.col('cse_distance').abs()))

pas_variants_ldf.select(upstream_tss=pl.col('cse_distance').min(), downstream_tss=pl.col('cse_distance').max()).collect()

upstream_tss,downstream_tss
i64,i64
-2149,2063


In [25]:
upstream=2000
downstream=2000

# filter out variants out of this range
pas_variants_ldf = pas_variants_ldf.filter((pl.col('cse_distance') >= -upstream) & (pl.col('cse_distance') <= downstream))

# join tss variants with individuals
pas_individual_variant_ldf = (variant_ldf.join(pas_variants_ldf, how='inner', on=['chrom', 'variant_start', 'variant_end', 'ref', 'alt']).
                              select([
                                  'individual', 'chrom', 'variant_start', 'variant_end', 'ref', 'alt',
                                  'gene', 'pas', 'strand', 'cse_distance', 'absolute_cse_distance', 'cse_pos'])
                             )

In [26]:
# keep the variant closest to the TSS for each individual and gene
# pas_individual_variant_ldf = pas_individual_variant_ldf.sort('absolute_cse_distance'). \
#     group_by(['individual', 'gene', 'chrom', 'strand']). \
#     agg(pl.col(['variant_start', 'variant_end', 'ref', 'alt', 'pas', 'cse_distance']).first())

pas_individual_variant_ldf = (pas_individual_variant_ldf
                              .with_columns(signed_cse_pos=pl.when(pl.col('strand') == '+').then(pl.col('cse_pos')).otherwise(-pl.col('cse_pos')))
                              .sort('signed_cse_pos', descending=True)
                              .group_by(['individual', 'gene', 'chrom', 'strand', 'ref', 'alt', 'variant_start', 'variant_end'])
                              .agg(pl.col(['pas', 'cse_distance']).first()))

# pas_individual_variant_ldf = pas_individual_variant_ldf. \
#     group_by(['individual', 'gene', 'chrom', 'strand', 'ref', 'alt', 'variant_start', 'variant_end']). \
#     agg(pl.col(['pas', 'cse_distance']).first())

# join outrider with variants on individual
pas_individual_variant_outrider_ldf = (
    pas_individual_variant_ldf
    .join(training_benchmark_ldf, how='inner', on=['individual', 'gene'])
    .select(['gene', 'tissue', 'individual', 'cse_distance', 'outlier_state', 
             'zscore', 'FDR', 'AbExp', *feature_columns]))

In [27]:
# pl.Config.set_streaming_chunk_size(100)
# print(veff_outrider_ldf.explain(streaming=True))

In [28]:
df = pas_individual_variant_outrider_ldf.collect()

In [29]:
df.shape

(15794160, 30)

In [30]:
df.head()

gene,tissue,individual,cse_distance,outlier_state,zscore,FDR,AbExp,expected_expr@theta,splice_ensemble@AbSplice,splice_ensemble@MMSplice_SpliceMap_Psi_ref,splice_ensemble@SpliceAI,vep@cadd_raw.max,vep@LoF_HC.proportion,vep@transcript_ablation.proportion,vep@stop_gained.proportion,vep@frameshift_variant.proportion,vep@coding_sequence_variant.proportion,vep@missense_variant.proportion,vep@inframe_deletion.proportion,vep@inframe_insertion.proportion,vep@stop_lost.proportion,vep@3_prime_UTR_variant.proportion,vep@5_prime_UTR_variant.proportion,vep@NMD_transcript_variant.proportion,vep@NMD_escaping_variant.proportion,vep@start_lost.proportion,vep@splice_donor_variant.proportion,vep@splice_acceptor_variant.proportion,vep@splice_region_variant.proportion
str,str,str,i64,enum,f32,f64,f64,f32,f64,f64,f64,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32
"""ENSG00000156482""","""Heart - Left Ventricle""","""GTEX-1GF9U""",-1693,"""normal""",0.281297,1.0,0.003678,636.466736,null,null,null,-0.20364,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""ENSG00000072756""","""Esophagus - Muscularis""","""GTEX-ZLFU""",-1501,"""normal""",0.656318,1.0,0.003434,479.290741,null,null,null,0.215677,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.086614,0.0,0.0,0.0,0.0,0.0
"""ENSG00000103544""","""Artery - Aorta""","""GTEX-T5JC""",-1116,"""normal""",0.833482,1.0,0.003991,547.123352,0.000006,null,0.0,-0.057801,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""ENSG00000118217""","""Whole Blood""","""GTEX-13FXS""",401,"""normal""",0.168664,1.0,0.000151,142.827362,0.000006,null,0.0,0.165801,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""ENSG00000112339""","""Lung""","""GTEX-15SB6""",289,"""normal""",2.450918,1.0,0.004213,463.492432,null,null,null,0.139572,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [31]:
df_bac = df

# cse_distance in bins
bin_size=50
cuts = list(range(-upstream + bin_size, downstream, bin_size))
cut_labels = [str(x) for x in [-upstream, *cuts]]
cse_distance_labels = {c: f'[{c}, {int(c) + bin_size})' for c in cut_labels}
df = (df.with_columns(cse_distance_bin=(pl.col('cse_distance').cut(cuts, labels=cut_labels))
                      .cast(pl.Enum(cut_labels)))
      .with_columns(is_underexpressed=(pl.col('outlier_state') == 'underexpressed')))

df = (df.with_columns(cse_distance_bin_label=pl.col("cse_distance_bin").replace_strict(cse_distance_labels), 
                                                      bin_size=pl.lit(50)))

In [32]:
df.head()

gene,tissue,individual,cse_distance,outlier_state,zscore,FDR,AbExp,expected_expr@theta,splice_ensemble@AbSplice,splice_ensemble@MMSplice_SpliceMap_Psi_ref,splice_ensemble@SpliceAI,vep@cadd_raw.max,vep@LoF_HC.proportion,vep@transcript_ablation.proportion,vep@stop_gained.proportion,vep@frameshift_variant.proportion,vep@coding_sequence_variant.proportion,vep@missense_variant.proportion,vep@inframe_deletion.proportion,vep@inframe_insertion.proportion,vep@stop_lost.proportion,vep@3_prime_UTR_variant.proportion,vep@5_prime_UTR_variant.proportion,vep@NMD_transcript_variant.proportion,vep@NMD_escaping_variant.proportion,vep@start_lost.proportion,vep@splice_donor_variant.proportion,vep@splice_acceptor_variant.proportion,vep@splice_region_variant.proportion,cse_distance_bin,is_underexpressed,cse_distance_bin_label,bin_size
str,str,str,i64,enum,f32,f64,f64,f32,f64,f64,f64,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,enum,bool,str,i32
"""ENSG00000156482""","""Heart - Left Ventricle""","""GTEX-1GF9U""",-1693,"""normal""",0.281297,1.0,0.003678,636.466736,null,null,null,-0.20364,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"""-1700""",false,"""[-1700, -1650)""",50
"""ENSG00000072756""","""Esophagus - Muscularis""","""GTEX-ZLFU""",-1501,"""normal""",0.656318,1.0,0.003434,479.290741,null,null,null,0.215677,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.086614,0.0,0.0,0.0,0.0,0.0,"""-1550""",false,"""[-1550, -1500)""",50
"""ENSG00000103544""","""Artery - Aorta""","""GTEX-T5JC""",-1116,"""normal""",0.833482,1.0,0.003991,547.123352,0.000006,null,0.0,-0.057801,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"""-1150""",false,"""[-1150, -1100)""",50
"""ENSG00000118217""","""Whole Blood""","""GTEX-13FXS""",401,"""normal""",0.168664,1.0,0.000151,142.827362,0.000006,null,0.0,0.165801,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"""400""",false,"""[400, 450)""",50
"""ENSG00000112339""","""Lung""","""GTEX-15SB6""",289,"""normal""",2.450918,1.0,0.004213,463.492432,null,null,null,0.139572,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"""250""",false,"""[250, 300)""",50


In [33]:
# set new bins
# new_bins = [(-2000, -500),
#             *[(i, i + 100) for i in range(-500, -100, 100)],
#             (-100, -50),
#             (-50, 0),
#             (0, 50),
#             (50, 100),
#             *[(i, i + 100) for i in range(100, 500, 100)],]
# new_bins = [(-2000, -1000),
#             (-1000, -500),
#             *[(i, i + 100) for i in range(-500, -100, 100)],
#             (-100, -50),
#             (-50, 0),
#             (0, 50),
#             (50, 100),
#             *[(i, i + 100) for i in range(100, 500, 100)],
#             (500, 1000),
#             (1000, 2000)]
new_bins = [(i, i + 250) for i in range(-2000, 2000, 250)]
new_bin_labels  = [f'[{start}, {stop})' for start, stop in new_bins]

df_bac = df
for start, stop in new_bins:
    df = df.with_columns(cse_distance_bin_label=(pl.when((pl.col('cse_distance_bin').cast(pl.Int16) >= start) & (pl.col('cse_distance_bin').cast(pl.Int16) <= stop))
                                                 .then(pl.lit(f'[{start}, {stop})'))                                  
                                                 .otherwise(pl.col('cse_distance_bin_label'))),
                         bin_size = (pl.when((pl.col('cse_distance_bin').cast(pl.Int16) >= start) & (pl.col('cse_distance_bin').cast(pl.Int16) < stop))
                                     .then(pl.lit(stop - start))
                                     .otherwise(pl.col('bin_size'))))

df = df.with_columns(cse_distance_bin_label=pl.col('cse_distance_bin_label').cast(pl.Enum(new_bin_labels)))

In [34]:
df.head()

gene,tissue,individual,cse_distance,outlier_state,zscore,FDR,AbExp,expected_expr@theta,splice_ensemble@AbSplice,splice_ensemble@MMSplice_SpliceMap_Psi_ref,splice_ensemble@SpliceAI,vep@cadd_raw.max,vep@LoF_HC.proportion,vep@transcript_ablation.proportion,vep@stop_gained.proportion,vep@frameshift_variant.proportion,vep@coding_sequence_variant.proportion,vep@missense_variant.proportion,vep@inframe_deletion.proportion,vep@inframe_insertion.proportion,vep@stop_lost.proportion,vep@3_prime_UTR_variant.proportion,vep@5_prime_UTR_variant.proportion,vep@NMD_transcript_variant.proportion,vep@NMD_escaping_variant.proportion,vep@start_lost.proportion,vep@splice_donor_variant.proportion,vep@splice_acceptor_variant.proportion,vep@splice_region_variant.proportion,cse_distance_bin,is_underexpressed,cse_distance_bin_label,bin_size
str,str,str,i64,enum,f32,f64,f64,f32,f64,f64,f64,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,enum,bool,enum,i32
"""ENSG00000156482""","""Heart - Left Ventricle""","""GTEX-1GF9U""",-1693,"""normal""",0.281297,1.0,0.003678,636.466736,null,null,null,-0.20364,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"""-1700""",false,"""[-1750, -1500)""",250
"""ENSG00000072756""","""Esophagus - Muscularis""","""GTEX-ZLFU""",-1501,"""normal""",0.656318,1.0,0.003434,479.290741,null,null,null,0.215677,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.086614,0.0,0.0,0.0,0.0,0.0,"""-1550""",false,"""[-1750, -1500)""",250
"""ENSG00000103544""","""Artery - Aorta""","""GTEX-T5JC""",-1116,"""normal""",0.833482,1.0,0.003991,547.123352,0.000006,null,0.0,-0.057801,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"""-1150""",false,"""[-1250, -1000)""",250
"""ENSG00000118217""","""Whole Blood""","""GTEX-13FXS""",401,"""normal""",0.168664,1.0,0.000151,142.827362,0.000006,null,0.0,0.165801,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"""400""",false,"""[250, 500)""",250
"""ENSG00000112339""","""Lung""","""GTEX-15SB6""",289,"""normal""",2.450918,1.0,0.004213,463.492432,null,null,null,0.139572,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"""250""",false,"""[250, 500)""",250


In [35]:
# calculate counts per bins
bin_count_df = (df.group_by(['outlier_state', 'cse_distance_bin_label', 'bin_size'])
                .agg((pl.len()).alias('count')))

# # calculate mean of each bin and then sum the means in each outlier state
totals_df = (bin_count_df.group_by('outlier_state').agg(pl.sum('count').alias('total_count')))
# # normalize each count by the mean calculated above
enrichment_df = (bin_count_df.join(totals_df, on='outlier_state')
                 .with_columns((pl.col('count') / pl.col('total_count')).alias('enrichment')))
ci_low, ci_high = sm.stats.proportion_confint(enrichment_df["count"], enrichment_df["total_count"])
enrichment_df = enrichment_df.with_columns(pl.Series(ci_low).alias('ci_low'), pl.Series(ci_high).alias('ci_high'))
# normalize by bin size
enrichment_df = enrichment_df.with_columns(enrichment = pl.col('enrichment') / pl.col('bin_size'),
                                           ci_low = pl.col('ci_low') / pl.col('bin_size'),
                                           ci_high = pl.col('ci_high') / pl.col('bin_size'))

In [36]:
enrichment_df

outlier_state,cse_distance_bin_label,bin_size,count,total_count,enrichment,ci_low,ci_high
enum,enum,i32,u32,u32,f64,f64,f64
"""overexpressed""","""[-500, -250)""",250,271,3334,0.000325,0.000288,0.000362
"""underexpressed""","""[-750, -500)""",250,381,5030,0.000303,0.000274,0.000332
"""normal""","""[1000, 1250)""",250,791529,15785796,0.000201,0.0002,0.000201
"""underexpressed""","""[-500, -250)""",250,353,5030,0.000281,0.000252,0.000309
"""underexpressed""","""[1250, 1500)""",250,129,5030,0.000103,0.000085,0.00012
…,…,…,…,…,…,…,…
"""overexpressed""","""[1750, 2000)""",250,163,3334,0.000196,0.000166,0.000225
"""normal""","""[1750, 2000)""",250,714314,15785796,0.000181,0.000181,0.000181
"""normal""","""[-250, 0)""",250,891263,15785796,0.000226,0.000225,0.000226


In [37]:
totals_df

outlier_state,total_count
enum,u32
"""overexpressed""",3334
"""normal""",15785796
"""underexpressed""",5030


In [38]:
base_path = Path('.')
df.write_parquet(base_path / 'pas_bin_abexp.parquet', use_pyarrow=True)
enrichment_df.write_parquet(base_path / 'pas_enrichment.parquet', use_pyarrow=True)
base_path

PosixPath('.')